<a href="https://colab.research.google.com/github/Vishu-Gupta/MLProjects/blob/main/01%20Kaggle%20Projects/06%20Forecast_Traffic_Flow_TPS_Mar'22/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This a Kaggle competition , part of the Mar'22 TPS.

Link: https://www.kaggle.com/c/tabular-playground-series-mar-2022

## 01 Linking with Kaggle and getting the datasets

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle competitions download tabular-playground-series-mar-2022

test.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
100% 4.69M/4.69M [00:00<00:00, 17.9MB/s]



In [5]:
!unzip train.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [6]:
!rm train.csv.zip # removing the zip file

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
data = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

## 02 Data Description
train.csv - the training set, comprising measurements of traffic congestion across 65 roadways from April through September of 1991.

row_id - a unique identifier for this instance

time - the 20-minute period in which each measurement was taken

x - the east-west midpoint coordinate of the roadway

y - the north-south midpoint coordinate of the roadway

direction - the direction of travel of the roadway. EB indicates 
"eastbound" travel, for example, while SW indicates a "southwest" direction of travel.

congestion - congestion levels for the roadway during each hour; the target. The congestion measurements have been normalized to the range 0 to 100.

test.csv - the test set; you will make hourly predictions for roadways identified by a coordinate location and a direction of travel on the day of 1991-09-30.

## 03 Data Preparation and EDA

In [9]:
data.shape # size of training data

(848835, 6)

In [10]:
df_test.shape # size of test data

(2340, 5)

In [11]:
data.head(20)

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60
5,5,1991-04-01 00:00:00,0,1,SB,58
6,6,1991-04-01 00:00:00,0,1,WB,26
7,7,1991-04-01 00:00:00,0,2,EB,31
8,8,1991-04-01 00:00:00,0,2,NB,49
9,9,1991-04-01 00:00:00,0,2,SB,46


In [12]:
data.nunique()

row_id        848835
time           13059
x                  3
y                  4
direction          8
congestion       101
dtype: int64

In [13]:
data.isnull().sum() # check for missing values

row_id        0
time          0
x             0
y             0
direction     0
congestion    0
dtype: int64

In [14]:
# no missing values.
# check for datatypes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848835 entries, 0 to 848834
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   row_id      848835 non-null  int64 
 1   time        848835 non-null  object
 2   x           848835 non-null  int64 
 3   y           848835 non-null  int64 
 4   direction   848835 non-null  object
 5   congestion  848835 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 38.9+ MB


In [15]:
#time is an object. Needs to be converted into datetime datatype
data['time'] = pd.to_datetime(data['time'])

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848835 entries, 0 to 848834
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   row_id      848835 non-null  int64         
 1   time        848835 non-null  datetime64[ns]
 2   x           848835 non-null  int64         
 3   y           848835 non-null  int64         
 4   direction   848835 non-null  object        
 5   congestion  848835 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 38.9+ MB


In [17]:
# for traffic the features given are the x, y points , and also the direction.
# in addition some important feature could be day_of_week , hour of day, the 20 minute interval of the data
# deriving these features

data['day_of_week'] = data['time'].dt.day_of_week
data['hour'] = data['time'].dt.hour
data['minute_interval'] = data['time'].dt.minute

In [18]:
data.nunique()

row_id             848835
time                13059
x                       3
y                       4
direction               8
congestion            101
day_of_week             7
hour                   24
minute_interval         3
dtype: int64

In [19]:
# keeping hours as numerical. Encoding all others

categ_cols = ['x','y','direction','day_of_week','minute_interval']
df = data
df = pd.get_dummies(df,columns=categ_cols,drop_first=True)

In [20]:
df.head()

,row_id,time,congestion,hour,x_1,x_2,y_1,y_2,y_3,direction_NB,direction_NE,direction_NW,direction_SB,direction_SE,direction_SW,direction_WB,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,minute_interval_20,minute_interval_40
0,0,1991-04-01,70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1991-04-01,49,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,1991-04-01,24,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,3,1991-04-01,18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,1991-04-01,60,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
features = list(df.columns)
features.remove('row_id')
features.remove('congestion')
features.remove('time')

In [22]:
len(features)

21

In [23]:
# convert the test data in same fashion
df_test['time'] = pd.to_datetime(df_test['time'])
df_test['day_of_week'] = df_test['time'].dt.day_of_week
df_test['hour'] = df_test['time'].dt.hour
df_test['minute_interval'] = df_test['time'].dt.minute
df_test = pd.get_dummies(df_test,columns=categ_cols,drop_first=True)

In [24]:
df_test.shape

(2340, 17)

In [25]:
features

['hour',
 'x_1',
 'x_2',
 'y_1',
 'y_2',
 'y_3',
 'direction_NB',
 'direction_NE',
 'direction_NW',
 'direction_SB',
 'direction_SE',
 'direction_SW',
 'direction_WB',
 'day_of_week_1',
 'day_of_week_2',
 'day_of_week_3',
 'day_of_week_4',
 'day_of_week_5',
 'day_of_week_6',
 'minute_interval_20',
 'minute_interval_40']

In [33]:
# since its only one day data , need to match with train
df_test['day_of_week_1'] = 0
df_test['day_of_week_2'] = 0
df_test['day_of_week_3'] = 0
df_test['day_of_week_4'] = 0
df_test['day_of_week_5'] = 0
df_test['day_of_week_6'] = 0

In [27]:
from sklearn.model_selection import train_test_split
df_train , df_validate = train_test_split(df,train_size=0.8,random_state=42)

## 04 Model building

In [28]:
import statsmodels.api as sm
model = sm.OLS(df_train['congestion'],sm.add_constant(df_train[features]))
results = model.fit()
results.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             congestion   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                 1.022e+04
Date:                Tue, 01 Mar 2022   Prob (F-statistic):               0.00
Time:                        19:08:52   Log-Likelihood:            -2.7864e+06
No. Observations:              679068   AIC:                         5.573e+06
Df Residuals:                  679046   BIC:                         5.573e+06
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 41.5297      0.084    492.121      0.000      41.364      41.695
hour                   0.3545      0.003    138.165      0.000       0.349       0.359
x_1                    7.2087      0.047    152.503      0.000       7.116       7.301
x_2                    7.1951      0.047    152.849      0.000       7.103       7.287
y_1                   -2.2911      0.055    -41.764      0.000      -2.399      -2.184
y_2                    3.9788      0.053     74.738      0.000       3.874       4.083
y_3                   -3.9003      0.053    -73.223      0.000      -4.005      -3.796
direction_NB           2.2778      0.058     38.953      0.000       2.163       2.392
direction_NE         -10.6695      0.069   -154.105      0.000     -10.805     -10.534
direction_NW         -26.1673      0.114   -230.504      0.000     -26.390     -25.945
direction_SB           4.1602      0.058     71.122      0.000       4.046       4.275
direction_SE         -18.5071      0.114   -162.523      0.000     -18.730     -18.284
direction_SW          -8.6212      0.069   -124.584      0.000      -8.757      -8.486
direction_WB          -0.1374      0.060     -2.292      0.022      -0.255      -0.020
day_of_week_1          0.6937      0.066     10.489      0.000       0.564       0.823
day_of_week_2          0.6162      0.066      9.315      0.000       0.487       0.746
day_of_week_3          0.7534      0.066     11.354      0.000       0.623       0.884
day_of_week_4          0.8931      0.066     13.508      0.000       0.764       1.023
day_of_week_5         -1.4413      0.066    -21.726      0.000      -1.571      -1.311
day_of_week_6         -3.3913      0.066    -51.286      0.000      -3.521      -3.262
minute_interval_20    -0.0006      0.044     -0.013      0.990      -0.086       0.085
minute_interval_40    -0.0013      0.044     -0.031      0.975      -0.087       0.084
==============================================================================
Omnibus:                     2981.851   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3018.200
Skew:                           0.162   Prob(JB):                         0.00
Kurtosis:                       2.960   Cond. No.                         108.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
# removing the minute_interval parameters as they have no dependency

features.remove('minute_interval_20')
features.remove('minute_interval_40')

# Model 2
model = sm.OLS(df_train['congestion'],sm.add_constant(df_train[features]))
results = model.fit()
results.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             congestion   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                 1.130e+04
Date:                Tue, 01 Mar 2022   Prob (F-statistic):               0.00
Time:                        19:08:54   Log-Likelihood:            -2.7864e+06
No. Observations:              679068   AIC:                         5.573e+06
Df Residuals:                  679048   BIC:                         5.573e+06
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            41.5290      0.081    515.475      0.000      41.371      41.687
hour              0.3545      0.003    138.166      0.000       0.349       0.359
x_1               7.2087      0.047    152.503      0.000       7.116       7.301
x_2               7.1951      0.047    152.849      0.000       7.103       7.287
y_1              -2.2911      0.055    -41.764      0.000      -2.399      -2.184
y_2               3.9788      0.053     74.738      0.000       3.874       4.083
y_3              -3.9003      0.053    -73.223      0.000      -4.005      -3.796
direction_NB      2.2778      0.058     38.953      0.000       2.163       2.392
direction_NE    -10.6695      0.069   -154.105      0.000     -10.805     -10.534
direction_NW    -26.1673      0.114   -230.504      0.000     -26.390     -25.945
direction_SB      4.1602      0.058     71.122      0.000       4.046       4.275
direction_SE    -18.5071      0.114   -162.523      0.000     -18.730     -18.284
direction_SW     -8.6212      0.069   -124.584      0.000      -8.757      -8.486
direction_WB     -0.1374      0.060     -2.292      0.022      -0.255      -0.020
day_of_week_1     0.6937      0.066     10.489      0.000       0.564       0.823
day_of_week_2     0.6162      0.066      9.315      0.000       0.487       0.746
day_of_week_3     0.7534      0.066     11.354      0.000       0.623       0.884
day_of_week_4     0.8931      0.066     13.508      0.000       0.764       1.023
day_of_week_5    -1.4413      0.066    -21.726      0.000      -1.571      -1.311
day_of_week_6    -3.3913      0.066    -51.286      0.000      -3.521      -3.262
==============================================================================
Omnibus:                     2981.857   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3018.206
Skew:                           0.162   Prob(JB):                         0.00
Kurtosis:                       2.960   Cond. No.                         108.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# go for complex models
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=42)
dt_base = dt.fit(df_train[features],df_train['congestion'])
df_train['dt_base_pred'] = dt_base.predict(df_train[features])
df_validate['dt_base_pred'] = dt_base.predict(df_validate[features])

In [31]:
from sklearn.metrics import mean_absolute_error
print('DT-Base : Training error is ',mean_absolute_error(df_train['congestion'],df_train['dt_base_pred']))
print('DT-Base : Validation error is ',mean_absolute_error(df_validate['congestion'],df_validate['dt_base_pred']))

DT-Base : Training error is  6.302402783719343
DT-Base : Validation error is  6.393968368193805


In [34]:
df_test['dt_base_pred'] = dt_base.predict(df_test[features])

In [35]:
df_test[['row_id','dt_base_pred']].rename(columns={'dt_base_pred':'congestion'}).to_csv('Submission dt_base.csv',index=False)

In [37]:
!kaggle competitions submit tabular-playground-series-mar-2022 -f 'Submission dt_base.csv' -m "Untuned Decision tree model"

100% 53.4k/53.4k [00:07<00:00, 7.10kB/s]
Successfully submitted to Tabular Playground Series - Mar 2022